## TO DO

- Delete empty images and add them to clean_dataset folder [DONE]
- Figure out how to shuffle by user, rather than by photo, make comparisons within users [DONE]
- Figure out how to do proper testing (code to pair up images randomly, predict their score using neural network, and then compare whether the ordinal match was right using our score) [DONE]

- Change training code to pull only images from new training set

### TO DO LATER
- Figure out how to incoroporate regression output
- Rewrite of code

## Create Neural Network

In [21]:
from keras import applications, optimizers
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import cv2
import numpy as np
import csv, sqlite3
import math
import os
import random

In [2]:
# If you want to specify input tensor
img_rows = 100
img_cols = 100
num_classes = 101

input_shape = (img_rows, img_cols, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# Custom Optimizer
opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=1e-6)

# Do not forget to compile it
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 49, 49, 64)        18496     
__________

## Train Neural Network

In [3]:
"""
Create dictionary with:
Key = filename root
Value = score
"""

con = sqlite3.connect("imagion.db")
cur = con.cursor()

table_cols = [i[0] for i in cur.execute("SELECT * FROM slimscoresclass").description]

def get_filenames(): 
    files_dict = {}
    
    cur.execute("SELECT filename, class_score FROM slimscoresclass")
    
    count = 0
    
    for file_, score in cur.fetchall():
        if count == 0:
            count += 1
            continue
        
        files_dict[file_] = score
            
        count += 1 
        
    return files_dict

files_dict = get_filenames()

In [4]:
"""
Create dictionary with:
Key = alias
Value = list of filenames for user
"""

DATASET_DIR = 'data_images'

def create_user_dict(dataset_dir):
    user_dict = {}
    
    for filename in os.listdir(dataset_dir):
        filename = filename.rsplit('.', 1)[0]
        alias = filename.rsplit('_', 1)[0]
        
        # do not include outliers
        if filename not in files_dict.keys():
            continue
            
        if alias not in user_dict:
            user_dict[alias] = [filename]
        else:
            user_dict[alias].append(filename)
            
    return user_dict

user_dict = create_user_dict(DATASET_DIR)        

In [5]:
"""Split training and test images"""

PERCENT_TRAINING = 0.75 

random.seed(10)
keys = user_dict.keys()

split = int(len(user_dict.keys()) * PERCENT_TRAINING)

random.shuffle(keys) # revisit this shuffle function

train_users = keys[:split]
test_users = keys[split:]

train_keys = []
test_keys = []

for user in train_users:
    for filename in user_dict[user]:
        train_keys.append(filename)
        
for user in test_users:
    for filename in user_dict[user]:
        test_keys.append(filename)
        
random.shuffle(train_keys)
random.shuffle(test_keys)

In [6]:
Y = np.arange(0, 10.1, 0.1)
Y = [str(num) for num in Y]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)

print encoder.classes_

['0.0' '0.1' '0.2' '0.3' '0.4' '0.5' '0.6' '0.7' '0.8' '0.9' '1.0' '1.1'
 '1.2' '1.3' '1.4' '1.5' '1.6' '1.7' '1.8' '1.9' '10.0' '2.0' '2.1' '2.2'
 '2.3' '2.4' '2.5' '2.6' '2.7' '2.8' '2.9' '3.0' '3.1' '3.2' '3.3' '3.4'
 '3.5' '3.6' '3.7' '3.8' '3.9' '4.0' '4.1' '4.2' '4.3' '4.4' '4.5' '4.6'
 '4.7' '4.8' '4.9' '5.0' '5.1' '5.2' '5.3' '5.4' '5.5' '5.6' '5.7' '5.8'
 '5.9' '6.0' '6.1' '6.2' '6.3' '6.4' '6.5' '6.6' '6.7' '6.8' '6.9' '7.0'
 '7.1' '7.2' '7.3' '7.4' '7.5' '7.6' '7.7' '7.8' '7.9' '8.0' '8.1' '8.2'
 '8.3' '8.4' '8.5' '8.6' '8.7' '8.8' '8.9' '9.0' '9.1' '9.2' '9.3' '9.4'
 '9.5' '9.6' '9.7' '9.8' '9.9']


In [7]:
def chunks(l, n):
    """Yield successive n-sized chunks from l"""
    for i in xrange(0, len(l), n):
        yield l[i:i+n]

def get_train_data(chunk, img_row, img_col):
    X_train = []
    Y_train = []
    
    for imgname in chunk:
        try:
            filename = 'data_images'+'/'+imgname+'.png'
            img = cv2.imread(filename)
            img = cv2.resize(img,(img_row,img_col))
            X_train.append(img)
            Y_train.append(str(files_dict[imgname]))
        except: 
            continue
    X_train = np.asarray(X_train)
    Y_train = np.asarray(Y_train)

    return X_train,Y_train
    
def get_test_data(chunk, img_row, img_col):
    X_test = []
    Y_test = []
    
    for imgname in chunk:
        try:
            filename = './data_images'+'/'+imgname+'.png'
            img = cv2.imread(filename)
            img = cv2.resize(img,(img_row,img_col))
            X_test.append(img)
            Y_test.append(str(files_dict[imgname]))
        except:
            continue
    X_test = np.asarray(X_test)
    Y_test = np.asarray(Y_test)

    return X_test,Y_test

def getTrainData(chunk,img_rows,img_cols):
    X_train,Y_train = get_train_data(chunk,img_rows,img_cols)
    if (X_train is not None and Y_train is not None):
        X_train/=255
        encoded_Y = encoder.transform(Y_train)
        Y_train=np_utils.to_categorical(encoded_Y, num_classes=101)
    return (X_train,Y_train)

def getTestData(chunk,img_rows,img_cols):
    X_test,Y_test = get_test_data(chunk,img_rows,img_cols)
    if (X_test is not None and Y_test is not None):
        X_test/=255
        encoded_Y = encoder.transform(Y_test)
        Y_test=np_utils.to_categorical(encoded_Y, num_classes=101)
    return (X_test,Y_test)

def test(model, nb_epoch, spatial_test_data, img_rows, img_cols):
    X_test,Y_test = getTestData(test_keys,img_rows,img_cols)
    return (X_test, Y_test)


In [8]:
num_epochs = 1
nb_epoch = 100
batch_size = 2
chunk_size = 32
img_rows = 100
img_cols = 100

In [ ]:
for e in range(nb_epoch):
    print('-'*40)
    print 'Epoch', e
    print('-'*40)
    print "Training..."
    instance_count=0


    for chunk in chunks(train_keys, chunk_size):
        X_chunk,Y_chunk=getTrainData(chunk,img_rows,img_cols)

        if (X_chunk is not None and Y_chunk is not None):
            loss = model.fit(X_chunk, Y_chunk, verbose=1, batch_size=batch_size, epochs=num_epochs)
            instance_count+=chunk_size
            
            print "Instance Count:", instance_count
            
            if instance_count%64==0:
                model.save_weights('model_class.h5',overwrite=True)

----------------------------------------
Epoch 0
----------------------------------------
Training...
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 32
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0625     
Instance Count: 64
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 96
(31, 101)

Epoch 1/1
31/31 [==============================] - 2s - loss: 0.0098 - acc: 0.0323     
Instance Count: 128
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0312     
Instance Count: 160
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 192
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 224
(32, 101)

Epoch 1/1
32/32 [========================

32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0312     
Instance Count: 2144
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 2176
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 2208
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 2240
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0625     
Instance Count: 2272
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 2304
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 2336
(32, 101)

Epoch 1/1
32/32 [==============================] - 2s - loss: 0.0098 - acc: 0.0000e+00     
Instance Count: 2368
(32, 101)

Epoch 1/1
32/32 [=========

## Load Weights

In [13]:
weights_path = 'model_class.h5'

if weights_path:
    model.load_weights(weights_path)

In [ ]:
""" SUDO CODE FOR CUSTOM ACCURACY FUNCTION"""

DATASET_DIR = 'data_images'

def test_accuracy():
    count = 0
    accuracy = 0
    
    for user in test_users:
        user_imgs = user_dict[user]
        
        if len(user_imgs) > 1:
           # get two elements from shuffled dictionary
            element1 = user_imgs.pop(0)
            element2 = user_imgs.pop(1)
            
            # read images
            img1 = cv2.imread(os.path.join(DATASET_DIR, element1+'.png'))
            img2 = cv2.imread(os.path.join(DATASET_DIR, element2+'.png'))
            
            #resize images
            img1 = cv2.resize(img1,(img_rows,img_cols))
            img2 = cv2.resize(img2,(img_rows,img_cols))
            
            # expand dimension
            img1 = np.expand_dims(img1, axis=0)
            img2 = np.expand_dims(img2, axis=0)
        
            # predict score for each image
            predict1 = np.argmax(model.predict(img1))
            predict2 = np.argmax(model.predict(img2))
            
            # compare whichever predicted score is higher
            
            # comparison dict
            if predict1 > predict2:
                max_predict = 'a'
            elif predict1 < predict2:
                max_predict = 'b'
            else:
                max_predict = 'equal'
            
            # compare whichever "score" is higher
            if files_dict[element1] > files_dict[element2]:
                max_actual = 'a'
            elif files_dict[element1] < files_dict[element2]:
                max_actual = 'b'
            else:
                max_actual = 'equal'

            # check if comparisons match 
            if max_predict == max_actual:
                accuracy +=1

            count += 1
            
            print max_predict
            print max_actual
            print max_predict == max_actual
            print '\n'
            
        else:
            continue
    
    accuracy_per = accuracy / count
    
    print count, "comparisons made"
    print "Accuracy score is:", accuracy_per
    
    return

test_accuracy()

## Predict Test

In [15]:
# 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16
img_names = ['xostylistxo_1',
             'xostylistxo_2', 
             'xostylistxo_3', 
             'xostylistxo_5', 
             'xostylistxo_6', 
             'xostylistxo_7', 
             'xostylistxo_9', 
             'xostylistxo_10', 
             'xostylistxo_11', 
             'xostylistxo_12', 
             'xostylistxo_13', 
             'xostylistxo_14',
             'xostylistxo_15',
             'xostylistxo_16']
             
for img_name in img_names:
    img = cv2.imread('data_images/'+img_name+'.png')
    img = cv2.resize(img,(160,160))
    img = img/255
    img = np.expand_dims(img, axis=0)

    print "Score is", model.predict(img)[0][0]

Score is 0.289618
Score is 0.501007
Score is 0.707981
Score is 0.501007
Score is 0.501007
Score is 0.501007
Score is 0.501007
Score is 0.501007
Score is 0.14877
Score is 0.501007
Score is 0.759573
Score is 0.501007
Score is 0.501007
Score is 0.548562


In [35]:
img2 = cv2.imread('data_images/'+img_name+'.png')
img2 = cv2.resize(img2,(160,160))
img2 = img2/255
img2 = np.expand_dims(img2, axis=0)

print "Score is", model.predict(img2)[0][0]

Score is 0.489362


In [20]:
for img_name in test_keys[:100]:
    img = cv2.imread('data_images/'+img_name+'.png')
    img = cv2.resize(img,(100,100))
    img = img/255
    img = np.expand_dims(img, axis=0)
    
    predictions = model.predict(img)
    top = np.argmax(predictions, axis=1)
    pred = encoder.inverse_transform([top])
    
    print "Score is", pred[0][0]

Score is 8.3
Score is 8.3
Score is 1.0
Score is 1.0
Score is 9.4
Score is 0.4
Score is 1.0
Score is 0.4
Score is 0.4
Score is 1.0
Score is 1.8
Score is 8.3
Score is 8.2
Score is 9.0
Score is 8.3
Score is 6.6
Score is 8.6
Score is 1.8
Score is 5.0
Score is 0.4
Score is 4.8
Score is 1.8
Score is 5.0
Score is 2.3
Score is 6.7
Score is 0.4
Score is 4.1
Score is 1.8
Score is 1.8
Score is 0.8
Score is 0.4
Score is 0.8
Score is 1.0
Score is 0.4
Score is 0.4
Score is 1.8
Score is 0.4
Score is 1.0
Score is 0.4
Score is 0.4
Score is 0.4
Score is 7.4
Score is 1.8
Score is 0.8
Score is 0.4
Score is 1.8
Score is 1.5
Score is 0.4
Score is 6.6
Score is 4.9
Score is 0.8
Score is 0.4
Score is 2.3
Score is 0.8
Score is 0.4
Score is 0.4
Score is 5.0
Score is 8.3
Score is 8.3
Score is 5.6
Score is 9.4
Score is 0.4
Score is 0.4
Score is 0.4
Score is 8.3
Score is 0.4
Score is 0.0
Score is 0.4
Score is 4.8
Score is 1.8
Score is 7.5
Score is 8.2
Score is 1.8
Score is 1.8
Score is 0.0
Score is 0.4
Score is 4.1